In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
IDir = r'/Volumes/PhD/imagery/masters/ROC/M1_ROC.shp'
ODir = r'/Volumes/PhD/imagery/masters/output/ROC/'
dNBR_df = gpd.read_file(IDir)
print(dNBR_df.columns)

Index(['BASIN_ID', 'Shape_Leng', 'Shape_Area', 'Segment_ID', 'Acc_Cl',
       'Perim_Buff', 'InsidePeri', 'OutsidePer', 'Fire_ID', 'Fire_Segme',
       'L_X1', 'L_X2', 'L_X3', 'V_X1', 'V_X2', 'Shape_Le_1', 'R', 'V_X3', 'X',
       'ExpX', 'P', 'PCl', 'PCl_Legend', 'LnV', 'Volume', 'VolMin', 'VolMax',
       'VolCl', 'VolCl_Lege', 'CombHaz', 'CombHazCl', 'CombHazCl_', 'area',
       'Field', 'ROC', 'nearest_st', 'distance_t', 'RI_index', 'RI',
       'Mean_dNBR', 'Med_dNBR', 'PF_dNBR', 'geometry'],
      dtype='object')


In [3]:
def M1(dnbr, Value):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    x_1 = dnbr['L_X1']
    x_3 = dnbr['L_X3']
    x_2 = dnbr[f'{Value}']
    R = dnbr['RI']
    
    lnx =  b + (b_1 * x_1 * R) + (b_2 * x_2 * R) + (b_3 * x_3 * R)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return prob
    

In [4]:
shp = r'/Volumes/PhD/imagery/masters/baer/thm2017_Segment_DFPredictions_15min_40mmh.shp'
shp = gpd.read_file(shp)
basin_id = 12042
basin_data = shp[shp['Segment_ID'] == basin_id]
print(basin_data['P'])

12424    0.414864
Name: P, dtype: float64


In [6]:
M1P_Mean = M1(dNBR_df, 'Mean_dNBR')
M1P_Median = M1(dNBR_df, 'Med_dNBR')
M1P_PF = M1(dNBR_df, 'PF_dNBR')
M1P_df = {
    'M1P_Mean': M1P_Mean,
    'M1P_Med': M1P_Median,
    'M1p_PF': M1P_PF
}
M1P_df = pd.DataFrame(M1P_df)

merged_gdf = gpd.GeoDataFrame(
    pd.merge(dNBR_df, M1P_df, left_index=True, right_index=True, how="left")
)


In [7]:
print(type(merged_gdf))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [11]:
merged_gdf.to_file(r'/Volumes/PhD/imagery/masters/ROC/M1_CT_ROC.shp')